In [1]:
import flwr as fl
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
edges = pd.read_csv("elliptic_txs_edgelist.csv")
features = pd.read_csv("elliptic_txs_features.csv",header=None)
classes = pd.read_csv("elliptic_txs_classes.csv")
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
features.columns = ["txId","time_step"] + tx_features + agg_features
features = pd.merge(features,classes,left_on="txId",right_on="txId",how='left')
features['class'] = features['class'].apply(lambda x: '0' if x == "unknown" else x)
features= features[features['class'] != '0' ]
def check(name):
  if(name in unique):
    return name 
  else : 
    return -1 
unique = features['txId'].unique()
edges["txId1"] = edges["txId1"].apply(lambda name: check(name))
edges["txId2"] = edges["txId2"].apply(lambda name: check(name))
edges = edges[edges["txId1"] != -1 ]
edges = edges[edges["txId2"] != -1 ]
class_values = sorted(features["class"].unique())
features_idx = {name: idx for idx, name in enumerate(sorted(features["txId"].unique()))}
features["txId"] = features["txId"].apply(lambda name: features_idx[name])
edges["txId1"] = edges["txId1"].apply(lambda name: features_idx[name])
edges["txId2"] = edges["txId2"].apply(lambda name: features_idx[name])
edges2 = edges[["txId1", "txId2"]].to_numpy().T
edge_weights = tf.ones(shape=edges2.shape[1])
node_features = tf.cast(
    features.sort_values("txId")[tx_features].to_numpy(), dtype=tf.dtypes.float32
)
graph_info = (node_features, edges2, edge_weights)

2023-12-28 23:45:51.468470: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-28 23:45:51.468724: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (karthik-HP-Pavilion-Laptop-15-eg2xxx): /proc/driver/nvidia/version does not exist
2023-12-28 23:45:51.471183: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.model_selection import train_test_split
X = features[tx_features + ['txId'] ]
y = features['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=117,shuffle=True)
x_train = X_train.txId.to_numpy()
x_test = X_test.txId.to_numpy()

In [4]:
model_path ="/home/karthik/final"
class GCNFlowerClient(fl.client.NumPyClient):
    def __init__(self, model_path):
        # Load the pre-trained GNN model
        self.gnn_model = tf.keras.models.load_model(model_path)
        print("Loaded model summary:")
        self.gnn_model.summary()

    def get_parameters(self, config):
        # Get the current parameters of the GNN model
        parameters = self.gnn_model.get_weights()
        print("Number of parameters:", len(parameters))
        return parameters

    def fit(self, parameters, config):
        self.gnn_model.set_weights(parameters)
        print("Number of weights in parameters:", len(parameters))
        
        # Local training on the client side using local data
        self.gnn_model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test), verbose=0)
        
        # Return the updated weights after local training
        return self.gnn_model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        self.gnn_model.set_weights(parameters)
        loss, accuracy = self.gnn_model.evaluate(x_test, y_test)
        return loss, len(x_test), {"accuracy": float(accuracy)}


In [5]:
flower_client = GCNFlowerClient(model_path)

Loaded model summary:
Model: "gnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocess (Sequential)     (None, 32)                4564      
                                                                 
 graph_conv1 (GraphConvLayer  multiple                 5888      
 )                                                               
                                                                 
 graph_conv2 (GraphConvLayer  multiple                 5888      
 )                                                               
                                                                 
 postprocess (Sequential)    (None, 32)                2368      
                                                                 
 logits (Dense)              multiple                  66        
                                                                 
Total params: 18,774
Trainable para

In [6]:
fl.client.start_numpy_client(
    server_address="127.0.0.1:5010",
    client=flower_client
)

INFO flwr 2023-12-28 23:45:58,539 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-12-28 23:45:58,579 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-12-28 23:45:58,587 | connection.py:42 | ChannelConnectivity.READY


Number of parameters: 74
Number of weights in parameters: 74


DEBUG flwr 2023-12-28 23:46:02,459 | connection.py:139 | gRPC channel closed


ValueError: in user code:

    File "/home/karthik/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/karthik/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/karthik/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/karthik/.local/lib/python3.10/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/karthik/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "gnn_model" (type GNNNodeClassifier).
    
    Could not find matching concrete f